In [1]:
import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

#Declaring variable for sentiment analysis
sia = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Fuad\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
import csv
import pandas as pd

# stock market lexicon
stock_lex = pd.read_csv('stock_lex.csv')
stock_lex['sentiment'] = (stock_lex['Aff_Score'] + stock_lex['Neg_Score'])/2
stock_lex = dict(zip(stock_lex.Item, stock_lex.sentiment))
stock_lex = {k:v for k,v in stock_lex.items() if len(k.split(' '))==1}
stock_lex_scaled = {}
for k, v in stock_lex.items():
    if v > 0:
        stock_lex_scaled[k] = v / max(stock_lex.values()) * 4
    else:
        stock_lex_scaled[k] = v / min(stock_lex.values()) * -4

# # Loughran and McDonald
# positive = []
# with open('lexicon_data/lm_positive.csv', 'r') as f:
#     reader = csv.reader(f)
#     for row in reader:
#         positive.append(row[0].strip())
    
# negative = []
# with open('lexicon_data/lm_negative.csv', 'r') as f:
#     reader = csv.reader(f)
#     for row in reader:
#         entry = row[0].strip().split(" ")
#         if len(entry) > 1:
#             negative.extend(entry)
#         else:
#             negative.append(entry[0])

final_lex = {}
# final_lex.update({word:2.0 for word in positive})
# final_lex.update({word:-2.0 for word in negative})
final_lex.update(stock_lex_scaled)
final_lex.update(sia.lexicon)
sia.lexicon = final_lex

In [3]:
import urllib
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pprint

In [4]:
page = urlopen('https://www.businesstimes.com.sg/search/brahim?page=').read()
soup = BeautifulSoup(page, features="html.parser")
posts = soup.findAll("div", {"class": "media-body"})# find the header

In [5]:
#headers_30 = [i.text for i in soup.find_all("div", {"class":"media-body"})]
for setr in posts:
    wort=setr.find("p").text
wort

"SATS Ltd has lobbed a conditional binding offer on Wednesday to buy a 49 per cent stake in a Malaysian airline catering firm from Bursa Malaysia-listed Brahim's Holdings Berhad for RM218 million (S$71 million). "

In [6]:
sentiment = sia.polarity_scores(wort)['compound']
sentiment

0.8073